# Building a Random Forest Classifier from Scratch

Data science lunch and learn -- 04/01/2022

It's surprisingly easy, and useful as a tutorial. Let's go!

## The Algorithm

A random forest decision tree classifier is a probabalistic extension of a vanilla decision tree. Decision trees use logical rules to parse features into finer and finer buckets, making decisions on the final nodes. This could look like a classifier for cats vs dogs with the following rules:

START --> POINTY EARS? --YES--> UNDER 20 LBS? --YES--> ...
                       \                      \--NO--> ...
                        \-NO--> UNDER 20 LBS? --YES--> ...
                         \                    \--NO--> ...
                         
                         
At the end of all our questions, we have a bin with a certain probability of being a cat or a dog. There are dogs with pointy ears (Doberman Pinschers come to mind), and their are cats bigger than 20 lbs (Maine Coons, or my mom's exceptionally fat shorthair), but odds are pretty good that an animal under 20 lbs with pointy ears is going to be a cat. We can add more rules for more and more specific binning (e.g., "wet nose vs dry nose?", "tail length > 30% body length?"), but you get the point.

Random forest classifiers differ from decision trees in a few simple ways. First, the rules are procedurally generated. This is also common in algorithmic decision trees, but it's worth mentioning in opposition to our toy example above. Given a sample of data with $n$ features, the algorithm will identify the feature that best distinguishes the classes and split the node, moving through multiple features or combinations of features to a given stop rule. 

Second, the algorithm is actually an *ensemble* of multiple decision trees. This derives from the wisdom of crowds phenomenon, which finds that a collection of weakly correlated learners will outperform a single strong predictor over time. Majority vote wins. So, if 600/1000 component trees say it's a cat, the algorithm will vote "cat".

Third, each component decision tree is made from a sample of the data, called a bag (aka bootstrap aggregation). 

Fourth, the algorithm enforces feature randomization. Each tree is limited by the set of features on which they are allowed to split a node.

These four innovations (probabalistic splitting, ensemble voting, bagged sampling, and feature randomization) lead to a surprisingly lightweight and robust classifier.


## Building a Random Forest

Taking that description, the basic outline of a random forest algorithm is

```
For each of N trees:
    - create a new bag (bootstrapped sample with replacementy) from the training set
    - use this bag to train a decision tree
    - at each node of the decision tree, randomly select *m* features, and select the optimal feature to divide the node (we'll use Shannon entropy but you can also use Gini impurity)
    - repeat to a stop rule OR all nodes complete
```


# Data

Let's use the relatively straightforward UCI Car Evaluation dataset. This uses six features to determine whether a car is a acceptable or unacceptable quality. This is a pretty standard toy dataset, although I can't really speak to it's quality. Let's go!

In [1]:
import pandas as pd
import numpy as np
import io
import requests

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'

In [3]:
s=requests.get(url).content
columns = ['buying','maint','doors','persons','lug_boot','safety', 'label']
df=pd.read_csv(io.StringIO(s.decode('utf-8')), header=None)
df.columns = columns
df.head()

,buying,maint,doors,persons,lug_boot,safety,label
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


### Formatting data types

In [4]:
cdict = {c:df[c].unique() for c in columns}

cdict

{'buying': array(['vhigh', 'high', 'med', 'low'], dtype=object),
 'maint': array(['vhigh', 'high', 'med', 'low'], dtype=object),
 'doors': array(['2', '3', '4', '5more'], dtype=object),
 'persons': array(['2', '4', 'more'], dtype=object),
 'lug_boot': array(['small', 'med', 'big'], dtype=object),
 'safety': array(['low', 'med', 'high'], dtype=object),
 'label': array(['unacc', 'acc', 'vgood', 'good'], dtype=object)}

In [5]:
bdict = {'vhigh':4, 'high':3, 'med':2, 'low':1}
df = df.replace({"buying": bdict})
# using the same dictionary
df = df.replace({"maint": bdict})

bdict = {'2':2, '3':3, '4':4, '5more':5}
df = df.replace({"doors": bdict})

bdict = {'2':2, '4':4, 'more':5}
df = df.replace({"persons": bdict})

bdict = {'small':1, 'med':2, 'big':3}
df = df.replace({"lug_boot": bdict})

bdict = {'low':1, 'med':2, 'high':3}
df = df.replace({"safety": bdict})

bdict = {'unacc':0, 'acc':1, 'good':2, 'vgood':3}
df = df.replace({"label": bdict})

In [6]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,label
0,4,4,2,2,1,1,0
1,4,4,2,2,1,2,0
2,4,4,2,2,1,3,0
3,4,4,2,2,2,1,0
4,4,4,2,2,2,2,0


In [7]:
df.dtypes

buying      int64
maint       int64
doors       int64
persons     int64
lug_boot    int64
safety      int64
label       int64
dtype: object

In [8]:
# multiclass problem
df.label.unique()

array([0, 1, 3, 2])

# Getting Started: The Basic Decision Tree

A random forest model is an expansion on a basic decision tree. The classic decision tree takes the following steps, which I've ordered in the class `dtree`:

1. read_data. We need to get the data and store it in the class. Since this is a fully supervised problem, we need complete data with features as columns and individual measures (here, cars) for every row. We also need labeled classes for each training sample.

2. make_tree. Loop through the data with the following steps at each loop:

2.1. calculate available features and classes to make a decision from.
2.2. calculate the default class (the most likely one on a blind guess) and the default entropy of the system. We'll use these to decide wherther a decision is better than chance.
2.3. find the best feature to make a decision. We do this by calculating the entropy (or Gini impurity) of make a decision on each of the available features. The feature that yields the most information is the winner, and is use to split the data.
2.4. add the decision point to the tree.
2.5. repeat steps 2.1-2.4 until no features remain.

3. Print the tree for review (this is kind of annoying so I've hacked something that works. There are much better alternatives for visualizing graphs out there in the world).

4. Predict holdout data for validaiton.

In [36]:
import numpy as np
import pandas as pd
from functools import partial

class dtree:
    """ A basic Decision Tree"""

    def __init__(self):
        """ Constructor """
        

    def read_data(self,filename):
        data = pd.read_csv('car_evaluation.csv')

        self.featureNames = data.columns[:-1].tolist()
        self.classes = data.label
        data = data[self.featureNames]
        
        return data, self.classes, self.featureNames
    

    def graph_depth(tree_dict):
        """
        Utility function for identifying tree (graph) depth
        """
        if isinstance(tree_dict, dict):

            return 1 + (max(map(dict_depth, tree_dict.values()))
                                        if tree_dict else 0)
        return 0
    

    def get_default_entropy(self, classes, newClasses, nData, metric='entropy'):
        """
        calculate baseline entropy and default class for a sample
        """
        information = 0
        frequency = pd.Series(newClasses).value_counts()

        for freq in frequency:
            if metric=='gini':
                information += self.calc_gini(freq/nData)
            else:
                information += self.calc_entropy(freq/nData)

        default = frequency.idxmax()

        return information, frequency, default
    
    
    def count_classes(self, classes):
        return np.unique(classes)
    

    def select_feature(self, information, data, featureNames, classes, forest = 0, metric='entropy'):
        """
        use entropy or gini impurity coefficient to identify best
        feature for decision to be made
        
        Parameters
        information
        data : list(list(int))
            data structure of feature variables
        featureNames : list(str)
            list of features to decide between
        classes : list(int)
            labels for data
        forest : int
            number of features to select at random for decision
            used when implementing decision tree as a random forest
        metric : str
            if 'entropy', uses Shannon entropy to decide most informative feature
            to make decision on
            if 'gini', uses Gini impurity
            
        Returns
        -------
        gain : float
            information gain
        bestFeature : str
            best feature to split decision tree
        """
        gain = np.zeros(len(featureNames))
#         featureSet = range(nFeatures)
        
        if forest != 0:
            np.random.shuffle(featureNames)
            featureNames = featureNames[0:forest]
        for f, feature in enumerate(featureNames):
            g = self.calc_info_gain(data,classes,feature)
            gain[f] = information - g

        bestFeature = featureNames[np.argmax(gain)]
        
        return gain, bestFeature
    
    
    def make_tree(self,data,classes,featureNames,maxlevel=-1,level=0,forest=0):
        """ 
        
        Make a decision tree!
        
        Parameters
        ----------
        data : pd.DataFrame
            n x m array of feature values for the dataset, without labels
            includes featureNames as columns
        classes : pd.Series
            n x 1 array of known target values for the dataset
        featureNames : list
            m-length list of feature names; subset of columns of data df
        maxlevel : int
            maximum depth of the tree
        level : int
            current level of the tree
            function will be used recursively to elevate level of the tree
        forest : int
            number of features to randomly exclude at each node
            if forest = 0, all features are used at every node
            
            e.g. forest = 2 and n_features=10, then 2 features
            will be randomly selected and 8 features excluded
            at every node.
        
        Returns
        -------
        tree : dict
            nested dictionary of graph structure
        """

        nData = data.shape[0]
        nFeatures = data.shape[1]

        try: 
            self.featureNames
        except:
            self.featureNames = featureNames

        # List the possible classes
        newClasses = self.count_classes(classes)

        # Compute the default class (and total entropy)
        information, frequency, default = self.get_default_entropy(classes, newClasses, nData)
        
        # find the best feature to branch

        if nData==0 or nFeatures == 0 or (maxlevel>=0 and level>maxlevel):
            # Have reached an empty branch
            return default
        elif len(np.unique(classes)) == 1:
            # Only 1 class remains
            return np.unique(classes)[0]
        else:            
            gain, bestFeature = self.select_feature(information, 
                                                    data, 
                                                    featureNames,
                                                    classes)
            
            tree = {bestFeature:{}}
            
            # recurse until complete
            # List the values that bestFeature can take
            values = []
            for idx, datapoint in data.iterrows():
                if datapoint[bestFeature] not in values:
                    values.append(datapoint[bestFeature])

            for value in values:

                newData = data[data[bestFeature] == value]
                newClasses = classes[data[bestFeature] == value]
                newNames = [f for f in featureNames if f != bestFeature]
                
                # Now recurse to the next level
                subtree = self.make_tree(data = newData,
                                         classes=newClasses,
                                         featureNames=newNames,
                                         maxlevel=maxlevel,
                                         level=level+1,
                                         forest=forest)

                # And on returning, add the subtree on to the tree
                tree[bestFeature][value] = subtree

            return tree        
            

    def printTree(self,tree,name=''):
        """
        Admittedly inelegant way to plot the tree
        """
        if type(tree) == dict:
            print(name, list(tree.keys())[0])
            for item in list(tree.values())[0].keys():
                print(name, item)
                self.printTree(list(tree.values())[0][item], name + "\t")
        else:
            print(name, "\t->\t", tree)
    

    def calc_entropy(self, p):
        out = -p * np.log2(p)
        return np.nan_to_num(out).sum()
    
        
    def calc_gini(self, p):
        p = np.array(p)
        return 1-sum((p**2))
    

    def calc_info_gain(self,data,classes,feature, metric='entropy'):
        """
        Calculates the information gain based on both entropy and the Gini impurity
        
        Parameters
        ----------
        data : pd.DataFrame
        
        classes : pd.Series
        
        feature : str
            feature name in data.columns
        
        metric : str
            if 'entropy', uses Shannon entropy to decide most informative feature
            to make decision on
            if 'gini', uses Gini impurity
        
        Returns
        -------
        gain : float
            information gain for a decision on a given feature
        """
        gain = 0
        nData = len(data)

        values = data[feature].unique()
        featureCounts = data[feature].value_counts()
        information = np.zeros(len(featureCounts))

        # Find where those values appear in data[feature] and the corresponding class
        for v, value in enumerate(values):
            newClasses = classes.loc[data[feature]==value]
            classValues = newClasses.unique()
            classCounts = newClasses.value_counts()

            if metric == 'gini':
                information[v] = self.calc_gini(classCounts / np.sum(classCounts))
            else:
                information[v] = self.calc_entropy(classCounts / np.sum(classCounts))
            
            gain += featureCounts[value]/nData * information[v]
            
        return gain

    
    def graph_depth(self, tree_dict):
        """
        Utility function for identifying tree (graph) depth
        """
        if isinstance(tree_dict, dict):

            return 1 + (max(map(self.graph_depth, tree_dict.values()))
                                        if tree_dict else 0)
        return 0
    

    def predict(self, tree, datapoint, verbose=False):
        """
        predict a single user's class

        Parameters
        ----------
        tree : dict
            graph structure of the decision tree
        datapoint : pd.Series
            row of data from a dataframe containing an individual user's data
            with feature names on the index

        Returns
        -------
        y_hat : pd.Series
            pandas Series of predictions
            may contain Nonetype values if no graph exists for a user's data
        """
        for key, val in datapoint.iteritems():
            if key in tree.keys():
                if verbose:
                    print(f'index: {datapoint.name} {key}: {val}')
                try:
                    tree = tree[key][val]
                except:
                    tree = None
                if isinstance(tree, dict):
                    return self.predict(tree, datapoint, verbose)
                else:
                    return tree

    def predictAll(self, tree, data):    
        """
        bulk predict on a pandas dataframe of values

        Parameters
        ----------
        tree : dict
            graph structure of the decision tree
        data : pd.DataFrame
            dataframe with users on the index and feature as labeled colums

        Returns
        -------
        y_hat : pd.Series
            pandas Series of predictions
            may contain Nonetype values if no graph exists for a user's data
        """
        f = partial(self.predict, tree)
        return data.apply(f, axis=1)



In [37]:
dt = dtree()

data, classes, featureNames = dt.read_data('car_evaluation.csv')

shuff = np.arange(len(data))
np.random.shuffle(shuff)
train_idx = shuff[:-200]
test_idx = shuff[-200:]

train = data.loc[train_idx]
test =  data.loc[test_idx]

train_classes = classes[train_idx]
test_classes = classes[test_idx]

dt.classes = train_classes

In [38]:
data.head()

,buying,maint,doors,persons,lug_boot,safety
0,4,4,2,2,1,1
1,4,4,2,2,1,2
2,4,4,2,2,1,3
3,4,4,2,2,2,1
4,4,4,2,2,2,2


In [39]:
tree = dt.make_tree(train, train_classes, featureNames)

In [40]:
# check depth of graph
dt.graph_depth(tree)

12

In [41]:
# plot graph structure
dt.printTree(tree)

 safety
 2
	 persons
	 4
		 buying
		 2
			 maint
			 4
				 lug_boot
				 3
					 	->	 1
				 1
					 	->	 0
				 2
					 doors
					 3
						 	->	 0
					 4
						 	->	 1
					 5
						 	->	 1
					 2
						 	->	 0
			 2
				 	->	 1
			 3
				 lug_boot
				 2
					 doors
					 2
						 	->	 0
					 3
						 	->	 0
					 5
						 	->	 1
					 4
						 	->	 1
				 1
					 	->	 0
				 3
					 	->	 1
			 1
				 lug_boot
				 1
					 	->	 1
				 3
					 	->	 2
				 2
					 doors
					 2
						 	->	 1
					 3
						 	->	 1
					 5
						 	->	 2
		 4
			 maint
			 1
				 lug_boot
				 1
					 	->	 0
				 2
					 doors
					 2
						 	->	 0
					 4
						 	->	 1
					 5
						 	->	 1
					 3
						 	->	 0
				 3
					 	->	 1
			 3
				 	->	 0
			 4
				 	->	 0
			 2
				 lug_boot
				 2
					 doors
					 4
						 	->	 1
					 2
						 	->	 0
					 5
						 	->	 1
					 3
						 	->	 0
				 3
					 	->	 1
				 1
					 	->	 0
		 1
			 maint
			 2
				 lug_boot
				 1
					 	->	 1
				 3
					 

### Verifying training performance

Decision trees are deterministic. We should not see any null values in their prediction. Classification of the training set should be 100% accurate.

In [42]:
y_hat = dt.predictAll(tree, train)

In [43]:
y_hat.isnull().sum()

0

In [44]:
from sklearn.metrics import classification_report, accuracy_score

In [45]:
# perfect reproduction of training data
print(classification_report(train_classes, y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1076
           1       1.00      1.00      1.00       344
           2       1.00      1.00      1.00        54
           3       1.00      1.00      1.00        54

    accuracy                           1.00      1528
   macro avg       1.00      1.00      1.00      1528
weighted avg       1.00      1.00      1.00      1528



### Testing set performance

Notably, decision trees cannot classify a combination of features they've never seen before. As a result, we're going to have some null values for weird edge causes (a 5-door care with 0 safety rating and a weird lug boot? Probably only one of those). 

There are two ways to hand that. One is to just admit that you cannot predict edge cases, and return null. This tends to make our data look better than it really is.

In [46]:
y_hat = dt.predictAll(tree, test)
y_hat.name = 'predictions'

In [47]:
results = pd.concat([y_hat, test_classes], axis=1)

In [48]:
results_nonull = results.dropna(axis=0)

In [49]:
print(f'accuracy is {accuracy_score(results_nonull.label, results_nonull.predictions):.3f}')
print(f'chance is {sum(np.array(results_nonull.label) == results_nonull.label.value_counts().idxmax())/len(results_nonull.label):.3f}')

accuracy is 0.956
chance is 0.731


In [50]:
print(classification_report(results_nonull.label, results_nonull.predictions))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       133
           1       0.83      0.97      0.89        30
           2       0.90      0.90      0.90        10
           3       0.88      0.78      0.82         9

    accuracy                           0.96       182
   macro avg       0.90      0.90      0.90       182
weighted avg       0.96      0.96      0.96       182



The other option is to fill null values with the most likely prediction (the default). As we can see, this doesn't work that well, since the most common prediction by definition has the most complete data, and therefore is rarely the answer we're looking for.

In [51]:
results_fillnull = results.fillna(results_nonull.label.value_counts().idxmax(),axis=0)

In [52]:
print(f'accuracy is {accuracy_score(results_fillnull.label, results_fillnull.predictions):.3f}')
print(f'chance is {sum(np.array(results_fillnull.label) == results_fillnull.label.value_counts().idxmax())/len(results_fillnull.label):.3f}')

accuracy is 0.875
chance is 0.670


In [53]:
print(classification_report(results_fillnull.label, results_fillnull.predictions))

              precision    recall  f1-score   support

           0       0.88      0.97      0.93       134
           1       0.83      0.72      0.77        40
           2       0.90      0.60      0.72        15
           3       0.88      0.64      0.74        11

    accuracy                           0.88       200
   macro avg       0.87      0.73      0.79       200
weighted avg       0.87      0.88      0.87       200



# Random Forest

Let's expand the decision tree algorithm to the random forest algorithm.

Basically, the RF algorithm extends the classic decision tree with two upgrades:

1. generate samples. Rather than create one deceision tree from all of the training data, RF genrates samples from the training data to create "weak" learners. 
2. forest features. Similarly, each tree is trained off of a subset of features to a max depth.
3. predictions are generated by taking the mode of the predictions from every graph produced.

In [122]:
class randomforest(dtree):
    """
    A simple random forest algorithm, expanded from a decision tree algorithm.
    """
    def __init__(self):
        """ Constructor """
        # call super()
        super().__init__()
        self.dtree = dtree()

        
    def rf(self,data,classes,features,nTrees,nSamples,nFeatures,maxlevel=-1):
        """
        Random forest loop.
        
        Parameters
        ----------
        data : pd.DataFrame
            dataframe of values with features on columns and measurement intervals on rows
            class target labels are not on this dataframe
        classes : pd.Series
            target class labels for the data
        features : list
            list of features to train the algorithm. All features must be rows in data
        nTrees : int
            number of random trees to generate
        nSamples : int
            number of samples to randomly select per each tree in the random forest
        nFeatures : int
            number of features to randomly select per each tree in the random forest
        maxlevel : int
            maximum depth of the decision tree
            -1 leads to max depth
        
        Returns
        -------
        tree_list : list(dict)
            list of n graphs, where n=nTrees        
        """
        tree_list = []

        for i in range(nTrees):
            print(i)
            # Compute bootstrap samples
            sample = data.sample(n=nSamples)
            sampleTarget = classes.loc[sample.index]

            tree_list.append(self.dtree.make_tree(sample,
                                                  sampleTarget,
                                                  featureNames,
                                                  maxlevel=maxlevel,
                                                  forest=nFeatures))
    
        return tree_list
    
    
    def rfpredict(self, tree_list, data):
        """
        predict class through majority voding of all trees generated by the random forest algorithm
        
        Parameters
        ----------
        tree_list : list(dict)
            list of graphs producted by iterative generation of decision trees generated by rf
        data : pd.DataFrame
            df of values with features on columns and measures per row
            
        Returns
        -------
        decision : pd.Series
            array of majority-vote predictions
            may contain null values for edge cases
        """
        # Majority voting
        yhats = pd.DataFrame()
        for t, tree in enumerate(tree_list):
            yhats[f'tree_{t}'] = dt.predictAll(tree, data)

        decision = yhats.mode(axis=1)
        if decision.shape[1] > 1:
            decision = decision[0] 
                
        return decision

In [123]:
rf = randomforest()

In [136]:
len(featureNames)

6

In [138]:
tree_list = rf.rf(train, train_classes, features=featureNames, nTrees=50, nSamples=len(train)//3, nFeatures=4)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


### Training classification
Random forests are less deterministic (but still pretty close). That little bit of stocasticity means that training samples will not be perfectly fit. This is a very good thing -- overfitting is bad.

In [139]:
train_preds = rf.rfpredict(tree_list, train)

In [141]:
print(classification_report(train_classes, train_preds))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1076
           1       0.96      0.99      0.98       344
           2       0.92      0.91      0.92        54
           3       0.93      0.96      0.95        54

    accuracy                           0.98      1528
   macro avg       0.95      0.96      0.96      1528
weighted avg       0.99      0.98      0.99      1528



### Testing classification


In [142]:
test_preds = rf.rfpredict(tree_list, test)

In [145]:
# with enough iterations of random forests, we greatly reduce the possibility of null values
test_preds.isnull().sum()

0

In [143]:
print(classification_report(test_classes, test_preds))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97       134
           1       0.74      0.93      0.82        40
           2       0.50      0.47      0.48        15
           3       0.44      0.36      0.40        11

    accuracy                           0.88       200
   macro avg       0.67      0.68      0.67       200
weighted avg       0.88      0.88      0.87       200



In [149]:
print(f'accuracy is {accuracy_score(test_classes, test_preds):.3f}')
print(f'chance is {sum(test_classes == test_classes.mode()[0])/len(test_classes):.3f}')

accuracy is 0.875
chance is 0.670


# Conclusions

Random forest algorithms are pretty powerful for being so conceptually simple. They still have many of the downfalls of linear classifiers, and become exponentially more complex with each new feature added. However, for simple classification they get the job done.